In [8]:
from azure.core.credentials_async import AsyncTokenCredential
# ! pip install agent-framework-azure-ai -U

In [9]:
import os

from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

from agent_framework import AgentRunResponse,ChatAgent,HostedFileSearchTool,HostedVectorStoreContent
from agent_framework.azure import AzureAIAgentClient

In [10]:
load_dotenv('../../.env')

True

In [11]:
async def create_vector_store(client: AzureAIAgentClient) -> tuple[str, HostedVectorStoreContent]:
    """Create a vector store with sample documents."""
    file_path = './document.md'
    agents_client = client.agents_client
    file = await agents_client.files.upload_and_poll(file_path=file_path, purpose="assistants")
    print(f"Uploaded file, file ID: {file.id}")


    vector_store = await agents_client.vector_stores.create_and_poll(file_ids=[file.id], name="graph_knowledge_base")

    print(f"Created vector store, ID: {vector_store.id}")


    return file.id, HostedVectorStoreContent(vector_store_id=vector_store.id)

In [12]:
async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=os.environ.get("AZURE_AI_AGENT_PROJECT_ENDPOINT"),
            model_deployment_name=os.environ.get("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME"),
            ) as chat_client,
    ):


        file_id, vector_store = await create_vector_store(chat_client)

        file_search = HostedFileSearchTool(inputs=vector_store)
        
        agent = chat_client.create_agent(
            name="PythonRAGDemo",
            instructions="""
                You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

                - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
                "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
                - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
                - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
                - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
                - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

                You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
                """,
            tools=[file_search],  # Tools available to the agent
            tool_choice = "auto",  # Let the agent decide when to use tools
        )
                

        print("Agent created. You can now ask questions about the uploaded document.")

        queries = [
            "Can you explain Contoso's travel insurance coverage?",
            "What are some popular destinations?",
        ]
        for query in queries:
            print("--------------------------------")
            print(f"\nUser question: {query}\n")
            print(f"Agent response: ", end="")
            async for chunk in agent.run_stream(query, tool_resources={"file_search": {"vector_store_ids": [vector_store.vector_store_id]}}):

                if chunk.text:
                    print(chunk.text, end="", flush=True)

Uploaded file, file ID: assistant-MhwbYBEyCv21gVw5C8Jkyg
Created vector store, ID: vs_R1pqVNQeBmmD2TCkfIepSf0d
Agent created. You can now ask questions about the uploaded document.
--------------------------------

User question: Can you explain Contoso's travel insurance coverage?

Agent response: Contoso’s travel insurance covers medical emergencies, trip cancellations, and lost baggage, as stated in the uploaded document .--------------------------------

User question: What are some popular destinations?

Agent response: Popular destinations mentioned in the uploaded document include the Maldives, the Swiss Alps, and African safaris .